# Climatiq notebook examples
[Climatiq](https://climatiq.io/) provides an API to help you estimate how much greenhouse gasses your activities emit, based on validated conversion factors known as “emission factors.”  
An emission factor is a way to convert different human activities into greenhouse gases.
It will take that human activity and estimate how much CO2 equivalent (abbreviated CO2e, usually expressed in kilograms), that activity will emit.

Different emission factors can help us answer questions like:
- How much CO2e is emitted taking the train from Paris to Berlin?
- How much CO2e is emitted spending $1,000 on soft drinks in the US?
- How much CO2e is emitted staying a night at a hotel in China?

Climatiq has a wide variety of emission factors available, that you can browse in our [Data Explorer](https://explorer.climatiq.io/), to get a feel for the sort of questions Climatiq can help you answer.

# API Key
All calls to the Climatiq API require that you have an account and that you send your API key. Signing up is free and takes only a few minutes.

[Click here](https://docs.climatiq.io/guides/getting-api-key) to see how to sign up and get your API key, which you should input below.

In [50]:
MY_API_KEY = "INSERT_YOUR_API_KEY_HERE" #TODO change this to your Climatiq API Key

In [51]:
# This block installs requests, which is a python library for doing http requests
# Install requests using pip in the current python kernel
# See https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/ for details
# Note that if you are using conda you might need to uncomment the later line
import sys
!{sys.executable} -m pip install requests
# Delete the above line, and uncomment the below line if you are using conda
# !conda install --yes --prefix {sys.prefix} numpy

import requests
import pprint
import json

Defaulting to user installation because normal site-packages is not writeable


# Documentation
This python notebook will mostly focus on some quick examples so you can get a feel for the API.
If you'd like a little more introduction, check our out [quickstart guide](https://docs.climatiq.io/guides/quickstart)

The notebook is tested using python3.10, but should work in python3.6 and above.

We also won't cover everything the API can do in this notebook, but check out the [API documentation](https://docs.climatiq.io/) for a lot more comprehensive documentation.

# Searching
If you need to figure out which emission factors are available, the easiest way is to try out our [Data Explorer](https://explorer.climatiq.io/) that allows you to search for, and filter emission factors based on sources, regions or free-text searching.

We also have a `search` rendpoint available for searching. It will let you do free-text searching, and optionally filtering on sources, regions etc.
See how to use it below.

In [52]:
url = "https://api.climatiq.io/data/v1/search"
query="grid mix"

query_params = {
    # Free text query can be writen under the query string
    "query": query,
    # You can also filter on region, year, source and more
    "region": "AT",
    # The data in the Climatiq API is versioned, to ensure
    # predictability of results. Read more about this in the docs.
    "data_version": "^9",
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

response = requests.get(url, params=query_params, headers=authorization_headers).json()
# The search endpoint returns a lot of stuff - here are the keys
print(response.keys())

# The most relevant is probably the results - here are the first three results
response["results"][0:3]

dict_keys(['current_page', 'last_page', 'total_results', 'results', 'possible_filters'])


[{'activity_id': 'electricity-supply_grid-source_residual_mix',
  'id': '9cc29d60-228c-4e33-89f8-b422b70c6ee9',
  'name': 'Electricity supplied from grid - residual mix',
  'category': 'Electricity',
  'sector': 'Energy',
  'source': 'GHG Protocol',
  'source_link': 'https://ghgprotocol.org/sites/default/files/GHG%20Emissions%20Calculation%20Tool_0.xlsx',
  'source_dataset': 'GHG Emissions Calculation Tool',
  'uncertainty': None,
  'year': 2021,
  'year_released': 2021,
  'region': 'AT',
  'region_name': 'Austria',
  'description': 'Emission intensity of Austria grid mix as reported. Published by the Association of Issuing Bodies (AIB). Retrieved from European Residual Mixes 2018 v1.2 (published July 2019) - Table 2 - Direct GWP (gCO2/kWh). NOTE: this value is provided in CO2 only and does not include the impact of other GHGs. WARNING: The primary source does not report an emission factor for Austria on Table 2 in European Residual Mixes 2018 v1.2. This emission factor is retrieved fr

# Estimates
Climatiq allows you to perform estimates based on an emission factor. You specify the emission factor using an activity id, and optionally filtering on region, year or other things.
The following code snippet will make an estimation of how much CO2e the power grid in Austria emits, for a certain amount of power generated. 
We will be using the `/estimate` endpoint. For a more comprehensive documentation about this endpoint, [click here](https://docs.climatiq.io/#estimate)

In [53]:
# The id for the emission factor. Found via the data explorer or the search endpoint
activity_id = "electricity-supply_grid-source_residual_mix"
# We have many regions with the same activity id, representing the power grid in different countries.
# We'd like to get the power for Australia specifically, so we will need to specify a region. We do this by specifying the UN location code for the region
# You can also see the region for different emission factors in the data explorer
region = "AT"

# We must also specify how much electricity generation we would like to make the estimation for.
# In this case we will do it for 100kilowattt-hours. 
# Different emission factors have different requirement as to what units they support estimates for
# You can see the units supported by an emission factor in the data explorer, and find more documentation about units
# in the API documentation.
parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

json_body = {
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
        "data_version": "^9"
        # More filters are possible here. See the full documentation for more options
    },
    # Specify how much energy we're estimating for
    "parameters": parameters
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the estimate endpoint with a json body and the correct authorization headers
response = requests.post("https://api.climatiq.io/data/v1/estimate", json=json_body, headers=authorization_headers)
# The response is also json.
pprint.pprint(response.json())



{'activity_data': {'activity_unit': 'kWh', 'activity_value': 100.0},
 'audit_trail': 'selector',
 'co2e': 54.59,
 'co2e_calculation_method': 'ar5',
 'co2e_calculation_origin': 'source',
 'co2e_unit': 'kg',
 'constituent_gases': {'ch4': None,
                       'co2': 54.59,
                       'co2e_other': None,
                       'co2e_total': 54.59,
                       'n2o': None},
 'emission_factor': {'access_type': 'public',
                     'activity_id': 'electricity-supply_grid-source_residual_mix',
                     'category': 'Electricity',
                     'data_quality_flags': ['partial_factor',
                                            'notable_methodological_variance'],
                     'id': '9cc29d60-228c-4e33-89f8-b422b70c6ee9',
                     'name': 'Electricity supplied from grid - residual mix',
                     'region': 'AT',
                     'source': 'GHG Protocol',
                     'source_dataset': 'GHG Emiss

# Estimate response
As you can see above, the estimate response is a JSON response that consists of three parts:
- The outermost part, which is the emissions for your estimation
- The `constituent_gases` which is additional information the source provides (if any) about what other greenhouse gases are emitted. You shouldn't worry too much about this field unless you know you need this.
- The `emission_factor`, which is Climatiq specifying what emission factor was used. You can see that the `id` and the `region` matches what we specified in our API call.


# Batch estimate
The Climatiq API is very fast, but if you have a lot of data it might be problematic to make an API call for every estimation. Climatiq has a batch endpoint where you can make up to 100 estimations per API call.
It will return a result for each estimation in the same order that the estimates were sent.
[More details about the batch estimate endpoint here](https://docs.climatiq.io/#batch-estimations)

Below we will make three estimations with the same emission factor, but different energy amounts

In [54]:
activity_id = "electricity-supply_grid-source_residual_mix"
region = "AT"
data_version = "^9"

parameters = {
        "energy": 100,
        "energy_unit": "kWh"
    }

# Notice that the JSON body here is an array, with each entry being an estimate
json_body = [
{
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
        "data_version": data_version
    },
    "parameters": {
        "energy": 10,
        "energy_unit": "kWh"
    }
},
{
    "emission_factor": {
        "activity_id": activity_id,
        "region": region,
        "data_version": data_version
    },
    "parameters": {
        "energy": 100,
        "energy_unit": "kWh"
    }
}
]

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

# We send a POST request to the /batch endpoint with a json body and the correct authorization headers
response = requests.post("https://api.climatiq.io/data/v1/estimate/batch", json=json_body, headers=authorization_headers)
# The response is also json.
response.json()



{'results': [{'co2e': 5.459,
   'co2e_unit': 'kg',
   'co2e_calculation_method': 'ar5',
   'co2e_calculation_origin': 'source',
   'emission_factor': {'name': 'Electricity supplied from grid - residual mix',
    'activity_id': 'electricity-supply_grid-source_residual_mix',
    'id': '9cc29d60-228c-4e33-89f8-b422b70c6ee9',
    'access_type': 'public',
    'source': 'GHG Protocol',
    'source_dataset': 'GHG Emissions Calculation Tool',
    'year': 2021,
    'region': 'AT',
    'category': 'Electricity',
    'source_lca_activity': 'electricity_generation',
    'data_quality_flags': ['partial_factor',
     'notable_methodological_variance']},
   'constituent_gases': {'co2e_total': 5.459,
    'co2e_other': None,
    'co2': 5.459,
    'ch4': None,
    'n2o': None},
   'activity_data': {'activity_value': 10.0, 'activity_unit': 'kWh'},
   'audit_trail': 'selector'},
  {'co2e': 54.59,
   'co2e_unit': 'kg',
   'co2e_calculation_method': 'ar5',
   'co2e_calculation_origin': 'source',
   'emissio

# Industry classification estimate

If your data comes with an industry specification code Climatiq API allows you to query emission factors, based on their NACE or ISIC industry classification codes. At the moment this endpoint works only with spend-based datasets, in particular EXIOBASE and GHG Protocol. 
[More details about the industry classification estimate endpoint here](https://www.climatiq.io/docs#classifications) 

In [55]:
url = "https://api.climatiq.io/classifications/v1/estimate"

data = {
        "classification": {
        "classification_type": "isic4", # here add your classification type, at the momement it will be either nace2 or isic4
        "classification_code": "25", # here add a code from your classification, e.g. for NACE 01.19 you would add 01.19 - note that the value has to be a string
        "source": "EXIOBASE", # choose the desired source - at them momement, only EXIOBASE and GHG Protocol data are mapped to the industry classification codes
        "year": 2019, # choose the year of interest
        "region": "DE" # choose your location of interest, double digit UN Locode or one of the pre-defined location codes - global from GHG Protocl or one of the 5 pre-defined 
        # "rest of the world locations" from EXIOBASE, see documentation around OEFDB for more information
    },
    "parameters": {
        "money": 3000,  # provide amount of money spend on the specific activity
        "money_unit": "usd" # choose currency, currently we support more than 30 currencies - please check the documentation
    }
}

# You must always specify your AUTH token in the "Authorization" header like this.
authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

response = requests.post(url,headers=authorization_headers,json=data) # this establishes connection to the API
response_data = json.loads(response.text) # this line queries the data

import pandas as pd
pd.json_normalize(response_data) # you could create a pandas library dataframe out of the API response 


,co2e,co2e_unit,co2e_calculation_method,co2e_calculation_origin,audit_trail,emission_factor.name,emission_factor.activity_id,emission_factor.id,emission_factor.access_type,emission_factor.source,...,emission_factor.category,emission_factor.source_lca_activity,emission_factor.data_quality_flags,constituent_gases.co2e_total,constituent_gases.co2e_other,constituent_gases.co2,constituent_gases.ch4,constituent_gases.n2o,activity_data.activity_value,activity_data.activity_unit
0,688.2,kg,ar5,source,enabled,Fabricated metal products/except machinery and...,metal_products-type_fabricated_metal_products_...,e321a1d8-442a-4d07-8034-8249a2a4a455,public,EXIOBASE,...,Fabricated Metal Products,unknown,[],688.2,None,None,None,None,2682.0,eur


# Moving on
Climatiq has much more to offer, such as:
- Over 12.000 emission factors covering purchsaes, fuel, accomodation, different travel modes, waste processing [and more](https://docs.climatiq.io/#examples).
- Estimating based on [flight routes](https://docs.climatiq.io/#flights)
- Estimating the emissions of [cloud compute usage](https://docs.climatiq.io/#cloud-computing)
- Estimating by using industry classification codes such as ISIC or NAICS